# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# city data file
city_data_path="output_data/cities.csv"

# Read csv and put into a dataframe
city_df=pd.read_csv(city_data_path)

In [3]:
city_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mataura,-46.1927,168.8643,75.00,52.0,100.0,10.00,NZ,1.611696e+09
1,1,samarai,-10.6167,150.6667,82.24,74.0,100.0,6.96,PG,1.611696e+09
2,2,port alfred,-33.5906,26.8910,69.01,95.0,88.0,1.01,ZA,1.611696e+09
3,3,egvekinot,66.3167,-179.1667,-15.38,87.0,62.0,12.24,RU,1.611696e+09
4,4,atuona,-9.8000,-139.0333,78.35,76.0,3.0,16.55,PF,1.611696e+09


In [4]:
city_renamed_df=city_df.drop(labels='Unnamed: 0', axis=1)

In [5]:
city_renamed_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,mataura,-46.1927,168.8643,75.00,52.0,100.0,10.00,NZ,1.611696e+09
1,samarai,-10.6167,150.6667,82.24,74.0,100.0,6.96,PG,1.611696e+09
2,port alfred,-33.5906,26.8910,69.01,95.0,88.0,1.01,ZA,1.611696e+09
3,egvekinot,66.3167,-179.1667,-15.38,87.0,62.0,12.24,RU,1.611696e+09
4,atuona,-9.8000,-139.0333,78.35,76.0,3.0,16.55,PF,1.611696e+09


In [6]:
city_clean_df=city_renamed_df.dropna(how="any")

In [7]:
city_clean_df.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,577.000000,577.000000,577.000000,577.000000,577.000000,577.000000,5.770000e+02
mean,21.231488,16.255691,49.487036,73.816291,56.471404,8.784489,1.611696e+09
std,32.898584,89.533550,33.439398,19.352206,39.876192,6.348424,1.043276e+02
min,-54.800000,-179.166700,-54.670000,6.000000,0.000000,0.180000,1.611695e+09
25%,-5.400000,-59.783300,30.200000,65.000000,15.000000,3.690000,1.611696e+09
50%,24.767200,19.234500,61.720000,78.000000,75.000000,7.000000,1.611696e+09
75%,49.816700,90.220100,75.990000,87.000000,96.000000,12.240000,1.611696e+09
max,78.218600,179.316700,93.200000,100.000000,100.000000,36.910000,1.611696e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
import gmaps
import gmaps.datasets
gmaps.configure(api_key=g_key)

# gmaps.figure()

In [9]:
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2) #how to avoid repeating map behavior?
heatmap_layer = gmaps.heatmap_layer(
    city_clean_df[["Lat", "Lng"]], weights=city_clean_df["Humidity"])


fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
heatmap_layer.max_intensity = 100
heatmap_layer.point_radius =7

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
city_renamed_df.head()
city_renamed_df.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,582.000000,582.000000,582.000000,582.000000,582.000000,582.000000,5.820000e+02
mean,20.849146,16.246489,49.674261,73.747423,56.352234,8.816770,1.611696e+09
std,33.014983,89.147630,33.358910,19.384411,39.847878,6.370663,1.039167e+02
min,-54.800000,-179.166700,-54.670000,6.000000,0.000000,0.180000,1.611695e+09
25%,-6.640025,-59.529150,30.307500,65.000000,15.000000,3.717500,1.611696e+09
50%,24.075000,18.385900,62.040000,78.000000,75.000000,7.000000,1.611696e+09
75%,49.187375,88.293725,75.960000,87.000000,95.750000,12.322500,1.611696e+09
max,78.218600,179.316700,93.200000,100.000000,100.000000,36.910000,1.611696e+09


In [28]:
temp_filter=(city_renamed_df['Max Temp']<78.0) & (city_renamed_df['Max Temp']>68.0)
cloud_filter=(city_renamed_df['Cloudiness']>10.00) & (city_renamed_df['Cloudiness']<25.00)
wind_filter=(city_renamed_df['Wind Speed']<6.00)

# cloud_filter=
ideal_weather_df=city_renamed_df.loc[temp_filter&cloud_filter&wind_filter,:]
display(ideal_weather_df)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
46,bambous virieux,-20.3428,57.7575,75.99,88.0,20.0,4.61,MU,1.611696e+09
71,quatre cocos,-20.2078,57.7625,75.99,88.0,20.0,4.61,MU,1.611696e+09
95,yeppoon,-23.1333,150.7333,75.20,83.0,20.0,5.75,AU,1.611695e+09
131,mahebourg,-20.4081,57.7000,73.40,88.0,20.0,4.61,MU,1.611696e+09
330,walvis bay,-22.9575,14.5053,69.01,82.0,15.0,5.75,NaN,1.611696e+09
360,sembe,1.6481,14.5806,72.88,85.0,17.0,3.09,CG,1.611696e+09
393,souillac,-20.5167,57.5167,73.40,88.0,20.0,4.61,MU,1.611696e+09
453,bolobo,-2.1667,16.2333,73.78,88.0,18.0,1.14,CD,1.611696e+09
481,puri,19.8000,85.8500,71.60,83.0,20.0,4.61,IN,1.611696e+09
525,bahile,9.9995,118.7878,72.45,83.0,18.0,1.14,PH,1.611696e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
hotel_df=ideal_weather_df.reset_index(drop=True)
display(hotel_df.head())


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,bambous virieux,-20.3428,57.7575,75.99,88.0,20.0,4.61,MU,1.611696e+09
1,quatre cocos,-20.2078,57.7625,75.99,88.0,20.0,4.61,MU,1.611696e+09
2,yeppoon,-23.1333,150.7333,75.20,83.0,20.0,5.75,AU,1.611695e+09
3,mahebourg,-20.4081,57.7000,73.40,88.0,20.0,4.61,MU,1.611696e+09
4,walvis bay,-22.9575,14.5053,69.01,82.0,15.0,5.75,NaN,1.611696e+09


In [31]:
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,bambous virieux,-20.3428,57.7575,75.99,88.0,20.0,4.61,MU,1.611696e+09,
1,quatre cocos,-20.2078,57.7625,75.99,88.0,20.0,4.61,MU,1.611696e+09,
2,yeppoon,-23.1333,150.7333,75.20,83.0,20.0,5.75,AU,1.611695e+09,
3,mahebourg,-20.4081,57.7000,73.40,88.0,20.0,4.61,MU,1.611696e+09,
4,walvis bay,-22.9575,14.5053,69.01,82.0,15.0,5.75,NaN,1.611696e+09,
5,sembe,1.6481,14.5806,72.88,85.0,17.0,3.09,CG,1.611696e+09,
6,souillac,-20.5167,57.5167,73.40,88.0,20.0,4.61,MU,1.611696e+09,
7,bolobo,-2.1667,16.2333,73.78,88.0,18.0,1.14,CD,1.611696e+09,
8,puri,19.8000,85.8500,71.60,83.0,20.0,4.61,IN,1.611696e+09,
9,bahile,9.9995,118.7878,72.45,83.0,18.0,1.14,PH,1.611696e+09,


In [32]:
# do a loop:
hotel_lat_list=[]
hotel_lng_list=[]

base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"


for idx, row in hotel_df.iterrows():
    print(f'Searching for hotels in {row["City"]}')
    location=str(row['Lat'])+","+str(row['Lng'])
#     print(location)
    query_params={
        "key":g_key,
        "location": location, #"latitude,longitude",
        "radius":5000,
        "type": "lodging",
        "language":"en"

    }

    response=requests.get(base_url,params=query_params)
    hotel_data=response.json()
    
    try:
        name=hotel_data['results'][0]['name']
        # do i need lat and long from df, or vicinity?
        address=hotel_data['results'][0]['vicinity']
        hotel_lat=hotel_data['results'][0]['geometry']['location']['lat']
        hotel_lng=hotel_data['results'][0]['geometry']['location']['lng']
        hotel_df.loc[idx, "Hotel Name"]=name
        hotel_lat_list.append(hotel_lat)
        hotel_lng_list.append(hotel_lng)
    except:
        print(f'Issue with {row["City"]}')
        hotel_df.loc[idx, "Hotel Name"]=np.nan
        hotel_lat_list.append(np.nan)
        hotel_lng_list.append(np.nan)

Searching for hotels in bambous virieux
Searching for hotels in quatre cocos
Searching for hotels in yeppoon
Searching for hotels in mahebourg
Searching for hotels in walvis bay
Searching for hotels in sembe
Issue with sembe
Searching for hotels in souillac
Searching for hotels in bolobo
Issue with bolobo
Searching for hotels in puri
Searching for hotels in bahile


In [33]:
hotel_df_clean=hotel_df.dropna()
display(hotel_df_clean)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,bambous virieux,-20.3428,57.7575,75.99,88.0,20.0,4.61,MU,1.611696e+09,Casa Tia Villa
1,quatre cocos,-20.2078,57.7625,75.99,88.0,20.0,4.61,MU,1.611696e+09,Emeraude Beach Attitude Hotel
2,yeppoon,-23.1333,150.7333,75.20,83.0,20.0,5.75,AU,1.611695e+09,Villa Mar Colina
3,mahebourg,-20.4081,57.7000,73.40,88.0,20.0,4.61,MU,1.611696e+09,Shandrani Beachcomber Resort & Spa
6,souillac,-20.5167,57.5167,73.40,88.0,20.0,4.61,MU,1.611696e+09,Shanti Maurice Resort & Spa
8,puri,19.8000,85.8500,71.60,83.0,20.0,4.61,IN,1.611696e+09,OYO 1577 Hotel Shree Hari Grand
9,bahile,9.9995,118.7878,72.45,83.0,18.0,1.14,PH,1.611696e+09,Hillside Resort Palawan


In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_clean.iterrows()]
locations = hotel_df_clean[["Lat", "Lng"]]

In [39]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))